# 3차 프로젝트: E-Commerce Data 분석

1,2차 프로젝트는 기본적인 분석하면서 전처리하고, 간단하게 분석하고, 통계적으로 분석해봤다면
3차는 시각화의 끝판왕이라고 부르는 plotly를 활용해서 시각화하는 프로젝트 진행,
4차는 strimly 간단하게 어플리케이션 만들 수 있는 패키지 활용해서 시각화한 화면을 대시보드로 구현 진행

## 데이터 출처: Kaggle(https://www.kaggle.com/datasets/benroshan/ecommerce-data?select=List+of+Orders.csv)
## 3개의 데이터 중 2개 데이터 활용

# (1) 데이터 로드 및 확인

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

Olist = pd.read_csv('/content/drive/MyDrive/List of Orders.csv')
Detail = pd.read_csv('/content/drive/MyDrive/Order Details.csv')

In [ ]:
Olist.head()

,Order ID,Order Date,CustomerName,State,City
0,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad
1,B-25602,01-04-2018,Pearl,Maharashtra,Pune
2,B-25603,03-04-2018,Jahan,Madhya Pradesh,Bhopal
3,B-25604,03-04-2018,Divsha,Rajasthan,Jaipur
4,B-25605,05-04-2018,Kasheen,West Bengal,Kolkata


Order ID: 주문번호  
Order Date: 주문날짜  
CustomerName: 고객명  
State, City: 배송지 정보

In [ ]:
Detail.head()

,Order ID,Amount,Profit,Quantity,Category,Sub-Category
0,B-25601,1275.0,-1148.0,7,Furniture,Bookcases
1,B-25601,66.0,-12.0,5,Clothing,Stole
2,B-25601,8.0,-2.0,3,Clothing,Hankerchief
3,B-25601,80.0,-56.0,4,Electronics,Electronic Games
4,B-25602,168.0,-111.0,2,Electronics,Phones


Order ID: 주문번호  
Amount: 판매가격  
Profit: 이익  
Quantity: 수량  
Category: 주문상품 카테고리  
Sub-Category: 주문상품 세부 카테고리

Olist에는 주문번호가 하나씩 있는데  
Detail에는 주문번호 같은게 중복적으로 있고 양, 카테고리 등이 나와있다.  
해석하면 Bharat이라는 사람이 Order Date에 총 4가지 카테고리에 있는 상품들을 구매했다는 의미이다.  

Olist : 장바구니 기준, 기준 당 하나씩 생김  
Detail: 영수증 혹은 장바구니 안에서 품목 하나 하나 나타낸 것  
영수증 Pos 데이터라고 한다.
Order ID 기준은 온라인에서는 장바구니 같은 개념이다. 오프라인은 영수증번호 같은 개념

In [ ]:
# Olist 데이터 형태 확인
Olist.info()
# 560개 데이터 중 500개만 널이 아니다라고 뜨니
# 아마도 공백이 존재할 거로 추측, 아무 값도 없는게 60개 있을 수 있다.
# Order Date는 datetype으로 바꿔주자.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560 entries, 0 to 559
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Order ID      500 non-null    object
 1   Order Date    500 non-null    object
 2   CustomerName  500 non-null    object
 3   State         500 non-null    object
 4   City          500 non-null    object
dtypes: object(5)
memory usage: 22.0+ KB


In [ ]:
# Detail 데이터 형태 확인
Detail.info()
# 1500개 데이터로 1500개 널이 아니다라고 뜬다.
# 단, profit이 마이너스인 형태가 있다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Order ID      1500 non-null   object 
 1   Amount        1500 non-null   float64
 2   Profit        1500 non-null   float64
 3   Quantity      1500 non-null   int64  
 4   Category      1500 non-null   object 
 5   Sub-Category  1500 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 70.4+ KB


In [ ]:
Detail.tail()
# Detail의 전체 데이터의 profit이 다 마이너스는 아니다.

,Order ID,Amount,Profit,Quantity,Category,Sub-Category
1495,B-26099,835.0,267.0,5,Electronics,Phones
1496,B-26099,2366.0,552.0,5,Clothing,Trousers
1497,B-26100,828.0,230.0,2,Furniture,Chairs
1498,B-26100,34.0,10.0,2,Clothing,T-shirt
1499,B-26100,72.0,16.0,2,Clothing,Shirt


In [ ]:
# 프로핏이 마이너스인 데이터 개수 파악하기
Detail[Detail['Profit']<0].shape
# 1500개 중 503개가 프로핏 마이너스이다.

(503, 6)

In [ ]:
Detail[Detail['Profit']>0].shape
# 프로핏이 플러스인거는 1500개 중 950개로
# 950+503 = 1453개
# 나머지 47개는 0인가?

(950, 6)

In [ ]:
Detail[Detail['Profit']==0].shape
# 47개는 프로핏 0 확인

(47, 6)

In [ ]:
# 그럼 Amount 도 확인해보자
Detail[Detail['Amount']>0].shape
# Amount는 0보다 작은 게 없다.

(1500, 6)

일반적으로 영수증 형태이다 보니 환불 이라는 게 있다 보니  
환불되면 보통 판매 금액과 이익 둘 다 마이너스 치나  
지금 이 경우는 판매금액은 플러스로 이익을 마이너스로 처리한 거로 추측  
이러한 처리 방법은 플랫폼이나 서비스마다 다르다.  
=> 프로핏이 마이너스는 환불된 경우  
=> 목적에 따라 분석 방법이 달라진다.  
환불은 중요하지 않고 고객이 구매했다는 사실이 중요할 때는  
마이너스 값을 상관 없이  
구매를 100원하면 -100으로 해서 산계처리를 더하면 0으로 하게 하는데  
우리는 -100은 상관없고 구매 100인게 중요해 하면 마이너스 데이터 드롭시키고 +값만 살려서 분석하거나  
총 더한 매출액이 중요할 경우나 실제 발생한 이익과 데이터가 똑같아야 할 경우  
더해서 0으로 처리해서 분석하기도 한다.  
=> 우리는 더해서 0으로 만들어서 분석해보기로 하자

# (2) 전처리

Olist 데이터에 null값 존재  
560개 데이터 중 columns 별 60개씩 결측치가 있다.  
60개가 다 모든 컬럼에 대해 NA 값으로 처리되어 있으면  
특정 컬럼만 잡아서 NA 값 다 삭제해 하면 되는데  
문제는 모든 컬럼에 대해서가 아니라 지그재그처럼 어떤 컬럼은 NA인데 다른 컬럼은 데이터가 있을 경우이다.

In [ ]:
Olist[Olist['Order ID'].isna()]
# 데이터가 작으니 눈으로 확인해도 되긴 하나 개수가 많을 경우

,Order ID,Order Date,CustomerName,State,City
500,NaN,NaN,NaN,NaN,NaN
501,NaN,NaN,NaN,NaN,NaN
502,NaN,NaN,NaN,NaN,NaN
503,NaN,NaN,NaN,NaN,NaN
504,NaN,NaN,NaN,NaN,NaN
505,NaN,NaN,NaN,NaN,NaN
506,NaN,NaN,NaN,NaN,NaN
507,NaN,NaN,NaN,NaN,NaN
508,NaN,NaN,NaN,NaN,NaN
509,NaN,NaN,NaN,NaN,NaN


In [ ]:
Olist[Olist['Order ID'].isna()].isnull()

,Order ID,Order Date,CustomerName,State,City
500,True,True,True,True,True
501,True,True,True,True,True
502,True,True,True,True,True
503,True,True,True,True,True
504,True,True,True,True,True
505,True,True,True,True,True
506,True,True,True,True,True
507,True,True,True,True,True
508,True,True,True,True,True
509,True,True,True,True,True


In [ ]:
Olist[Olist['Order ID'].isna()].isnull().sum()
# 균등하게 모든 컬럼이 다 결측치이다.
# 실제로는 함수를 만들어서 열별로 루프 돌려서 확인한다.

Order ID        60
Order Date      60
CustomerName    60
State           60
City            60
dtype: int64

In [ ]:
# Olist에서 Order ID는 중요한 컬럼이니 요걸로 확인해서 null값들 없애자.
Olist = Olist[Olist['Order ID'].isna()==False]
Olist.shape

(500, 5)

In [ ]:
# merge로 이너 조인
# 위에 결측치 제거 없이 그냥 merge해도 알아서 정리된다.
data = Olist.merge(Detail, on = 'Order ID')

In [ ]:
data.shape

(1500, 10)

또 확인해봐야 할 게  
결과적으로는 1500개 되었으나  
머지하면서 누락된게 있는지, 중복이 혹시 생기지 않았는지 꼭 확인한다.  

In [ ]:
# Olist 의 Order ID의 유니크한 값이 과 data의 Order ID의 유니크한 값이 같은지 확인해서 누락되었는지 확인하자.
len(Olist['Order ID'].unique())

500

In [ ]:
len(data['Order ID'].unique())
# 값이 같으므로 Olist 누락된 데이터는 없다.

500

In [ ]:
# Detail 의 Order ID의 유니크한 값이 과 data의 Order ID의 유니크한 값이 같은지 확인해서 누락되었는지 확인하자.
len(Detail['Order ID'].unique())
# 값이 같으므로 Detail에서 누락된 데이터는 없다.
# 이런 경우에는 중복될거도 특별히 없다.

500

In [ ]:
# 결측치도 확인해보자.
data.isnull().values.any()
# any()는 하나라도 True(결측치) 값이 있으면 False라도 안 뜸

False

In [ ]:
data.isnull()
# 판다스 형태로 결측치 확인할 수 있는데

,Order ID,Order Date,CustomerName,State,City,Amount,Profit,Quantity,Category,Sub-Category
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
1495,False,False,False,False,False,False,False,False,False,False
1496,False,False,False,False,False,False,False,False,False,False
1497,False,False,False,False,False,False,False,False,False,False
1498,False,False,False,False,False,False,False,False,False,False


In [ ]:
data.isnull().values
# numpy 어레이 형태로 된다.

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [ ]:
# Olist에  Detail 컬럼 잘 연결된거 확인
data.head()

,Order ID,Order Date,CustomerName,State,City,Amount,Profit,Quantity,Category,Sub-Category
0,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad,1275.0,-1148.0,7,Furniture,Bookcases
1,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad,66.0,-12.0,5,Clothing,Stole
2,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad,8.0,-2.0,3,Clothing,Hankerchief
3,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad,80.0,-56.0,4,Electronics,Electronic Games
4,B-25602,01-04-2018,Pearl,Maharashtra,Pune,168.0,-111.0,2,Electronics,Phones


In [ ]:
# 날짜 변환
data['Order Date'] = pd.to_datetime(data['Order Date'], format = '%d-%m-%Y')

In [ ]:
# 날짜가 언제부터 언제까지 인지 조회해보자.
data['Order Date'].min(), data['Order Date'].max()
# 1년 동안의 데이터

(Timestamp('2018-04-01 00:00:00'), Timestamp('2019-03-31 00:00:00'))

In [ ]:
# 년, 월 칼럼 추가
# 년도와 월을 합친 컬럼을 추가
# order date를 스트링으로 바꾼 다음에 0번 인덱스부터 7번인덱스 (2019-03) 까지로 해서 짜른다.(slice)
data['year'] = data['Order Date'].dt.year
data['month'] = data['Order Date'].dt.month
data['yearmonth'] = data['Order Date'].astype('str').str.slice(0,7)

In [ ]:
data.head()

,Order ID,Order Date,CustomerName,State,City,Amount,Profit,Quantity,Category,Sub-Category,year,month,yearmonth
0,B-25601,2018-04-01,Bharat,Gujarat,Ahmedabad,1275.0,-1148.0,7,Furniture,Bookcases,2018,4,2018-04
1,B-25601,2018-04-01,Bharat,Gujarat,Ahmedabad,66.0,-12.0,5,Clothing,Stole,2018,4,2018-04
2,B-25601,2018-04-01,Bharat,Gujarat,Ahmedabad,8.0,-2.0,3,Clothing,Hankerchief,2018,4,2018-04
3,B-25601,2018-04-01,Bharat,Gujarat,Ahmedabad,80.0,-56.0,4,Electronics,Electronic Games,2018,4,2018-04
4,B-25602,2018-04-01,Pearl,Maharashtra,Pune,168.0,-111.0,2,Electronics,Phones,2018,4,2018-04


In [ ]:
# 잘 바뀌었는지 확인해 보자.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Order ID      1500 non-null   object        
 1   Order Date    1500 non-null   datetime64[ns]
 2   CustomerName  1500 non-null   object        
 3   State         1500 non-null   object        
 4   City          1500 non-null   object        
 5   Amount        1500 non-null   float64       
 6   Profit        1500 non-null   float64       
 7   Quantity      1500 non-null   int64         
 8   Category      1500 non-null   object        
 9   Sub-Category  1500 non-null   object        
 10  year          1500 non-null   int32         
 11  month         1500 non-null   int32         
 12  yearmonth     1500 non-null   object        
dtypes: datetime64[ns](1), float64(2), int32(2), int64(1), object(7)
memory usage: 140.8+ KB


# (3) EDA

# Plotly를 이용한 EDA 시각화
# 판매 분석
# plotly 홈페이지 (https://plotly.com/python/)
- 특징: interactive 하다
- 그래프로 그리면 특정값 하나가 궁금한 값이 있으면 값이 먼지 알 수 없는데 plotly는 마우스 가져다 두면 바로 확인할 수 있고 그래프 상에서 확대하거나 축소, 다운로드 등 기능적으로 다양하게 사용할 수 있다.

In [ ]:
import plotly.express as px

In [ ]:
# 년월별 주문수량의 합계를 봐보자.
df = data.groupby('yearmonth').agg({'Quantity' : 'sum'})
df

,Quantity
yearmonth,
2018-04,389
2018-05,423
2018-06,369
2018-07,240
2018-08,446
2018-09,331
2018-10,419
2018-11,578
2018-12,412


In [ ]:
# 위대로만 하면 yearmonth 컬럼이 인덱스로 잡히니 인덱스를 만들어주자
df = df.reset_index()
df

,yearmonth,Quantity
0,2018-04,389
1,2018-05,423
2,2018-06,369
3,2018-07,240
4,2018-08,446
5,2018-09,331
6,2018-10,419
7,2018-11,578
8,2018-12,412
9,2019-01,745


# Line 그래프 그리기

In [ ]:
# 년월별 주문량 그래프로 보기
fig1 = px.line(df, x='yearmonth', y='Quantity', title='Sales Quantity by month')
fig1.show()
# 카메라 버튼 누르면 png 파일로 다운 받을 수도 있고 그래프 꼭지점 누르면 값도 나오고 확대 축소 할 수 있다.
# 시계열적으로 보면 증가하는 트렌드가 있다.

In [ ]:
# 년월별 판매금액 보기
df2 = data.groupby('yearmonth').agg({'Amount':'sum'}).reset_index()
fig2 = px.line(df2, x='yearmonth', y='Amount', title='Sales Amount by month')
fig2.show()
# 시계열적으로 보면 증가하는 트렌드가 있다.

# Bar 그래프 그리기

In [ ]:
# 앞에서는 groupby로 집계했는데 여기서는 pivot_table로 집계를 해보자.
# 카테고리별 수량 합계 보기
df3 = data.pivot_table(index = 'Category', values=['Quantity'], aggfunc = 'sum').reset_index()
df3

,Category,Quantity
0,Clothing,3516
1,Electronics,1154
2,Furniture,945


In [ ]:
fig3 = px.bar(df3, x = 'Category', y = 'Quantity')
fig3.show()

In [ ]:
# 년월별 카테고리별 주문수량 합계 보기
df4 = data.pivot_table(index=['yearmonth', 'Category'], values = 'Quantity', aggfunc = 'sum').reset_index()
df4.head()

,yearmonth,Category,Quantity
0,2018-04,Clothing,251
1,2018-04,Electronics,75
2,2018-04,Furniture,63
3,2018-05,Clothing,233
4,2018-05,Electronics,116


# 누적 Bar 그래프
- 그냥 bar 그래프보다 시계열적인 비교를 하기 위해 많이 유용하게 사용한다.

In [ ]:
# 누적 bar 그래프로 표현하기
# 년월별 카테고리별 수량 누적 바 그래프로 보기
fig4 = px.bar(df4, x = 'yearmonth', y = 'Quantity', color = 'Category')
fig4.show()
# 전체적인 수량 자체는 증가한다고 볼 수 있다

# heat map을 이용한 지역별 주력 판매상품 분석

In [ ]:
df5 = data.pivot_table(index = ['State', 'Sub-Category'], values = ['Quantity', 'Amount', 'Profit'],  aggfunc = 'sum').reset_index()
df5.head()

,State,Sub-Category,Amount,Profit,Quantity
0,Andhra Pradesh,Accessories,231.0,22.0,2
1,Andhra Pradesh,Bookcases,4581.0,-1642.0,25
2,Andhra Pradesh,Electronic Games,188.0,-193.0,2
3,Andhra Pradesh,Furnishings,57.0,7.0,3
4,Andhra Pradesh,Hankerchief,710.0,80.0,32


In [ ]:
# density_heatmap:  밀도에 따라 heatmap 그려짐
# z 가 밀도를 어떤거로 할 건지 정하는 것
# 주소별, 카테고리별 수량
fig5 = px.density_heatmap(df5, x = 'State', y = 'Sub-Category', z = 'Quantity', title = 'Quantity heat map')
fig5.show()
# 주소별, 카테고리별 판매금액
fig6 = px.density_heatmap(df5, x = 'State', y = 'Sub-Category', z = 'Amount', title = 'Amount heat map')
fig6.show()
# 많이 팔렸다고 해도 제품별 금액이 다르니 좀 다르게 나타난다.
# saree는 인도 여성 옷

# 4차 프로젝트: streamlit을 이용한 대쉬보드 그리기
# 주소 (https://streamlit.io/) - gallery - data visualization
- 파이썬으로 웹 어플리케이션 만드는 쉽고 간단한 방법 중 하나
- 시각화 해서 대시보드 만드는 형태로 사용하기에는 어려움이 없다.

In [ ]:
!pip install streamlit
# 코랩환경에서 웹을 띄우기 위한 모듈
!pip install localtunnel
!pip install "ipywidgets>=7, <8"

ERROR: Could not find a version that satisfies the requirement localtunnel (from versions: none)
ERROR: No matching distribution found for localtunnel
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.0 MB/s eta 0:00:00


# (1) 코드 함수 모듈화
- 중복코드 없애기 위해 진행



In [ ]:
def load_data():
  Olist = pd.read_csv('/content/drive/MyDrive/List of Orders.csv')
  Detail = pd.read_csv('/content/drive/MyDrive/Order Details.csv')
  data = Olist.merge(Detail, on = 'Order ID')

  return data

def preproc():
  data['Order Date'] = pd.to_datetime(data['Order Date'], format = '%d-%m-%Y')
  data['year'] = data['Order Date'].dt.year
  data['month'] = data['Order Date'].dt.month
  data['yearmonth'] = data['Order Date'].astype('str').str.slice(0,7)

  return df

In [ ]:
df = data.groupby('yearmonth').agg({'Quantity' : 'sum'})
df = df.reset_index()
fig1 = px.line(df, x='yearmonth', y='Quantity', title='Sales Quantity by month')
fig1.show()

df2 = data.groupby('yearmonth').agg({'Amount':'sum'}).reset_index()
fig2 = px.line(df2, x='yearmonth', y='Amount', title='Sales Amount by month')
fig2.show()

In [ ]:
def line_chart(data, x, y, title):
  df = data.groupby(x).agg({y :'sum'}).reset_index()
  fig = px.line(df, x=x, y=y, title=title)
  fig.show()

  return fig

In [ ]:
# 함수 잘 만들어 졌는지 확인해보기
fig1 = line_chart(data, 'yearmonth', 'Quantity', 'Sales Quantity by month')
fig2 = line_chart(data, 'yearmonth', 'Amount', 'Sales Amount by month')

In [ ]:
df3 = data.pivot_table(index = 'Category', values=['Quantity'], aggfunc = 'sum').reset_index()
df3
fig3 = px.bar(df3, x = 'Category', y = 'Quantity')
fig3.show()

df4 = data.pivot_table(index=['yearmonth', 'Category'], values = 'Quantity', aggfunc = 'sum').reset_index()
df4.head()
fig4 = px.bar(df4, x = 'yearmonth', y = 'Quantity', color = 'Category')
fig4.show()

In [ ]:
# 첫번째 차트는 그냥 bar 차트였는데
# 두번째 차트는 누적 bar 차트였어서
# index 부분 values 부분, x, y,color 값 다르다.
# 다른 부분들만 처리해주면 된다.
def bar_chart(data, x, y, color=None):
  if color is not None:
    index = [x, color]
  else:
    index =x

  df = data.pivot_table(index=index, values=y, aggfunc='sum').reset_index()
  fig = px.bar(df, x=x, y=y, color=color)
  fig.show()

  return fig

In [ ]:
# 함수 잘 만들었는지 확인해보자.
fig3 = bar_chart(data, 'Category', 'Quantity')
fig4 = bar_chart(data, 'yearmonth', 'Quantity', 'Category')

In [ ]:
df5 = data.pivot_table(index = ['State', 'Sub-Category'], values = ['Quantity', 'Amount', 'Profit'],  aggfunc = 'sum').reset_index()
df5.head()

fig5 = px.density_heatmap(df5, x = 'State', y = 'Sub-Category', z = 'Quantity', title = 'Quantity heat map')
fig5.show()

# 주소별, 카테고리별 판매금액
fig6 = px.density_heatmap(df5, x = 'State', y = 'Sub-Category', z = 'Amount', title = 'Amount heat map')
fig6.show()

In [ ]:
# heatmap 함수 만들기
def heatmap(data, z, title):

  df = data.pivot_table(index = ['State', 'Sub-Category'], values = ['Quantity', 'Amount', 'Profit'],  aggfunc = 'sum').reset_index()
  fig = px.density_heatmap(df, x = 'State', y = 'Sub-Category', z = z, title = title)
  fig.show()

  return fig

In [ ]:
# 잘 만들었는지 확인하기
fig5 = heatmap(data, 'Quantity', 'Quantity heat map')
fig6 = heatmap(data, 'Amount', 'Amount heat map')

# (2) streamlit을 이용한 대시보드 구현

In [ ]:
# 주피터노트북에서만 쓰는 커맨드로 %%
# app.py 파일을 만들어라 라는 의미
# 파일 안에 같은 셀에 있는 내용들이 들어간  py라는 파일을 만들어라 라는 의미
# .py 파일을 만드는 이유는
# 우리는 주피터노트북에서 만드는 환경이니까
# 주피터노트북은 서비스나 어플리케이션을 바로 실행시킬 수 있는 확장자는 아니다.
# 파이썬이 프로그램을 실행시킬 수 있는 .py 사용
# 셀 나누면 안됨

%%writefile app.py

import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np


# 데이터 로드
# 위에서 만든 함수들 다 붙여넣기

# @: python에서 데코레이터인데, 로드 데이터라는게 실행되는데 st.cache_data가 데코레이터 함수라고 표현하는데
# @st.cache_data를 하면 def st.cache_data 함수 안에 def load_data() 함수가 들어가 있는 형태로 실행된다.
# load_data() 실행되고 나서 결과가 st.cache_data() 함수로 가고 또 실행되는 되는 형태
# 파이썬에서 쓰는 방법 중 하나
# 쓰는 이유는 data가 csv 파일을 app 으로 불러와서 메모리에 로드가 될텐데
# 전처리를 하고 그래프를 만들어서 시각화해서 웹에 띄울건데
# 웹 화면을 보면 버튼이 2개 있다고 하면 버튼1은 line차트 그리는 버튼이고 버튼2는 bar차트 그리는 버튼이라고 하면
# 만약에 버튼1 누르면 표현 영역에 라인차트 그려지고 버튼2 누르면 표현영역에 있던게 사라지고 바차트가 보일텐데
# 라인을 맨 처음 그렸다가 바를 그리면 웹 어플리케이션 같은 경우에는
# csv 불러오고 시각화하고 웹 구현하고 작업을 하고
# 버튼2 눌러서 바를 하면 같은 데이터 임에도 불구하고 또 csv 불러오고 시각화하고 웹 구현하고 해야한다.
# 데이터를 불러오고 이런 작업이 꽤 시간이 걸리는 작업인데
# 웹에서는 페이지가 바뀔 때마다 데이터를 불러오니까 작업 시간이 오래 걸린다.
# 그래서 로드 작업의 결과를 cache 라는 메모리에 가지고 있게 해서
# A B 바뀔 때 마다 적용해서 쓰게끔 하는 거로 속도에 영향을 미치게 하기 위해서다.

@st.cache_data
def load_data():
  Olist = pd.read_csv('/content/drive/MyDrive/List of Orders.csv')
  Detail = pd.read_csv('/content/drive/MyDrive/Order Details.csv')
  data = Olist.merge(Detail, on = 'Order ID')

  return data

# 전처리
def preproc():
  data['Order Date'] = pd.to_datetime(data['Order Date'], format = '%d-%m-%Y')
  data['year'] = data['Order Date'].dt.year
  data['month'] = data['Order Date'].dt.month
  data['yearmonth'] = data['Order Date'].astype('str').str.slice(0,7)

  return data

# line 차트
def line_chart(data, x, y, title):
  df = data.groupby(x).agg({y :'sum'}).reset_index()
  fig = px.line(df, x=x, y=y, title=title)
  fig.show()

  return df, fig

# bar 차트
def bar_chart(data, x, y, color=None):
  if color is not None:
    index = [x, color]
  else:
    index =x

  df = data.pivot_table(index=index, values=y, aggfunc='sum').reset_index()
  fig = px.bar(df, x=x, y=y, color=color)
  fig.show()

  return fig

# heatmap 차트
def heatmap(data, z, title):

  df = data.pivot_table(index = ['State', 'Sub-Category'], values = ['Quantity', 'Amount', 'Profit'],  aggfunc = 'sum').reset_index()
  fig = px.density_heatmap(df, x = 'State', y = 'Sub-Category', z = z, title = title)
  fig.show()

  return fig

# 메인 함수 구현 - 어플리케이션 구동되는 부분
if __name__ == "__main__":

  # 제목
  st.title('E-Commerce Data 분석')
  # 내가 원하는 글씨 써진다. 타이틀이랑 글씨 크기 다름
  st.write('시각화 대시보드 만들기')

  # 데이터 로드
  data = load_data()
  # 데이터 전처리
  data = preproc()

# 서브 제목 만들기
st. subheader('월별 판매량 분석')

# 영역 나눠놓은 부분
with st.form('form', clear_on_submit = True):

  # 페이지를 columns 처럼 영역 나눔
  col1, col2 = st.columns(2)

  # 버튼을 만들어주는데 버튼 누르면 결과값 제출된게 변수에 저장
  submitted1 = col1.form_submit_button('판매량 그래프')
  submitted2 = col2.form_submit_button('매출액 그래프')

  if submitted1:
    df1, fig1 = line_chart(data, 'yearmonth', 'Quantity', 'Sales Quantity by month')
    # 위에서 만든 데이터프레임 형태가 길죽한 형태니까 transpose해서 가로로 만듦
    # 차트 노출
    st.dataframe(df1.T)
    # 그래프 노출
    st.plotly_chart(fig1, theme='streamlit', use_container_width=True)

  elif submitted2:
    df2, fig2 = line_chart(data, 'yearmonth', 'Amount', 'Sales Amount by month')
    st.dataframe(df2.T)
    st.plotly_chart(fig2, theme='streamlit', use_container_width=True)


st.subheader('품목별 판매량')
col1, col2 = st.columns(2)

with col1:
  col1.subheader('카테고리별 판매량')
  fig3 = bar_chart(data, 'Category', 'Quantity')
  st.plotly_chart(fig3, theme='streamlit', use_container_width=True)
with col2:
  col2.subheader('월별/카테고리별 누적 차트')
  fig4 = bar_chart(data, 'yearmonth', 'Quantity', 'Category')
  st.plotly_chart(fig4, theme='streamlit', use_container_width=True)


st.subheader('지역별 주력 판매상품')
# tab 이용, 클릭하는 거에 따라 달라짐
tab1, tab2 = st.tabs(['Quantity heat map', 'Amount heat map'])
with tab1:
  fig5= heatmap(data, 'Quantity', 'Quantity heat map')
  st.plotly_chart(fig5, theme='streamlit', use_container_width=True)
with tab2:
  fig6= heatmap(data, 'Amount', 'Amount heat map')
  st.plotly_chart(fig6, theme='streamlit', use_container_width=True)



Overwriting app.py


In [ ]:
!streamlit run /content/app.py & npx lt --port 8501

your url is: https://mighty-taxes-nail.loca.lt



  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.148.186.243:8501

2024-05-15 13:25:21.269 Serialization of dataframe to Arrow table was unsuccessful due to: ("Expected bytes, got a 'int' object", 'Conversion failed for column 0 with type object'). Applying automatic fixes for column types to make the dataframe Arrow-compatible.
2024-05-15 13:25:26.029 Serialization of dataframe to Arrow table was unsuccessful due to: ("Expected bytes, got a 'float' object", 'Conversion failed for column 0 with type object'). Applying automatic fixes for column types to make the dataframe Arrow-compatible.
2024-05-15 13:25:53.171 Serialization of dataframe to Arrow table was unsuccessful due to: ("Expected bytes, got a 'int' object", 'Conversion failed for column 0 with type object'). Applying automatic fixes for column types to make the dataframe Arrow-compatible.
2024-05-15 13:26:

# 브라우저 여는 법
위 코드 실행하면   
  Network URL: http://172.28.0.12:8501  
  External URL: http://34.148.186.243:8501  

your url is: https://thin-flowers-cover.loca.lt  
  
external URL 나오는데 이거 숫자 부분만 복사  
your url 클릭 - 매번 실행할 때마다 주소 다를 거다.  
tunnel password에   
34.148.186.243  
입력  
click to summit 클릭  
!streamlit run /content/app.py & npx lt --port 8501  
lt할 때 엘 L 임